In [33]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor
from joblib import dump
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam
from keras.metrics import MeanSquaredError, MeanAbsoluteError, MeanAbsolutePercentageError
from math import sqrt
import numpy as np
import scipy.stats as stats

In [34]:
# importing the data
file_paths = ("raw data/Market Prices.xls", "raw data/Market Prices 2.xls", "raw data/Market Prices 3.xls", "raw data/Market Prices 4.xls", "raw data/Market Prices 5.xls", "Raw Data/Market Prices 6.xls", "raw data/Market Prices 7.xls", "Raw Data/Market Prices 8.xls" )


dfs = []
for file in file_paths:
  df = pd.read_excel(file)
  dfs.append(df)

df = pd.concat(dfs)

df

,Commodity,Classification,Grade,Sex,Market,Wholesale,Retail,Supply Volume,County,Date
0,Dry Maize,Mixed-Traditional,-,-,Isebania Market,-,32.00/Kg,45000.0,Migori,2024-08-27
1,Dry Maize,White Maize,-,-,Ahero,45.00/Kg,50.00/Kg,7000.0,Kisumu,2024-08-27
2,Dry Maize,White Maize,-,-,Nyamakima,44.44/Kg,70.00/Kg,NaN,Nairobi,2024-08-27
3,Dry Maize,White Maize,-,-,Kathonzweni,35.00/Kg,40.00/Kg,7200.0,Makueni,2024-08-27
4,Dry Maize,White Maize,-,-,Kawangware,40.00/Kg,50.00/Kg,NaN,Nairobi,2024-08-27
...,...,...,...,...,...,...,...,...,...,...
1272,Dry Maize,-,-,-,Loitoktok Market,25.56/Kgs,35.00/Kgs,9900.0,Kajiado,2021-05-24
1273,Dry Maize,White Maize,-,-,Chepseon,24.56/Kgs,28.33/Kgs,1800.0,Kericho,2021-05-24
1274,Dry Maize,Yellow Maize,-,Male,Nakuru Wakulima,36.67/Kgs,40.00/Kgs,NaN,Nakuru,2021-05-24
1275,Dry Maize,White Maize,-,-,Elwak Market,40.00/Kgs,50.00/Kgs,1200.0,Mandera,2021-05-24


In [35]:
# Drop irrelevant columns
df.drop(['Commodity','Grade','Sex'], axis=1, inplace=True)


# Verify columns have been dropped
df.head()

,Classification,Market,Wholesale,Retail,Supply Volume,County,Date
0,Mixed-Traditional,Isebania Market,-,32.00/Kg,45000.0,Migori,2024-08-27
1,White Maize,Ahero,45.00/Kg,50.00/Kg,7000.0,Kisumu,2024-08-27
2,White Maize,Nyamakima,44.44/Kg,70.00/Kg,NaN,Nairobi,2024-08-27
3,White Maize,Kathonzweni,35.00/Kg,40.00/Kg,7200.0,Makueni,2024-08-27
4,White Maize,Kawangware,40.00/Kg,50.00/Kg,NaN,Nairobi,2024-08-27


In [36]:
df.replace(['-', ' - ', '- ', ' -'], np.nan, inplace=True)
df.isna().sum()

Classification     668
Market               0
Wholesale         1936
Retail             908
Supply Volume     4401
County               0
Date                 0
dtype: int64

In [37]:
df.shape

(22277, 7)

In [38]:
df.dropna(inplace=True)

In [40]:
df.shape

(16242, 7)

In [22]:
# Convert 'Wholesale' and 'Retail' to numerical values for exploration
price_columns = ["Wholesale", "Retail"]

for col in price_columns:
  df[col] = df[col].str.lower().str.replace("/kg", "").str.strip()
  df[col] = df[col].str.lower().str.replace("s", "").str.strip().astype(float)

df.head()

,Classification,Market,Wholesale,Retail,Supply Volume,County,Date
1,White Maize,Ahero,45.00,50.0,7000.0,Kisumu,2024-08-27
3,White Maize,Kathonzweni,35.00,40.0,7200.0,Makueni,2024-08-27
5,Mixed-Traditional,Nkubu,33.33,40.0,720.0,Meru,2024-08-27
6,White Maize,Kabiyet Market,45.00,50.0,1500.0,Nandi,2024-08-27
7,Mixed-Traditional,Kapsabet Market,35.56,36.0,10000.0,Nandi,2024-08-27


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16242 entries, 1 to 1276
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Classification  16242 non-null  object 
 1   Market          16242 non-null  object 
 2   Wholesale       16242 non-null  float64
 3   Retail          16242 non-null  float64
 4   Supply Volume   16242 non-null  float64
 5   County          16242 non-null  object 
 6   Date            16242 non-null  object 
dtypes: float64(3), object(4)
memory usage: 1015.1+ KB


In [24]:
# Sort data by 'Market' and 'Date'
df.sort_values(by=['County', 'Market','Classification', 'Date',], inplace=True)

In [25]:
df.head(20)

,Classification,Market,Wholesale,Retail,Supply Volume,County,Date
541,Mixed-Traditional,Eldama Ravine,27.78,40.00,1000.0,Baringo,2021-05-24
1055,Mixed-Traditional,Eldama Ravine,22.22,40.00,900.0,Baringo,2021-05-24
1809,Mixed-Traditional,Eldama Ravine,31.11,45.00,900.0,Baringo,2021-06-28
368,Mixed-Traditional,Eldama Ravine,31.11,32.00,900.0,Baringo,2021-09-20
3,Mixed-Traditional,Eldama Ravine,30.00,35.86,900.0,Baringo,2021-10-04
2534,Mixed-Traditional,Eldama Ravine,30.00,36.00,900.0,Baringo,2021-10-25
1176,Mixed-Traditional,Eldama Ravine,30.00,36.00,600.0,Baringo,2022-01-03
639,Mixed-Traditional,Eldama Ravine,30.00,34.00,1800.0,Baringo,2022-01-31
500,Mixed-Traditional,Eldama Ravine,30.00,34.00,1500.0,Baringo,2022-02-07
2406,Mixed-Traditional,Eldama Ravine,3.33,40.00,1200.0,Baringo,2022-04-04


In [26]:
# removing markets with less than 10 records
threshold = 10

# Filter out markets with less than 'threshold' records
market_counts = df["Market"].value_counts()
markets_to_keep = market_counts[market_counts >= threshold].index

# Keep only the data for markets with enough records
data = df[df['Market'].isin(markets_to_keep)]

# Check how many markets remain
print(f"Number of remaining markets: {data['Market'].nunique()}")

Number of remaining markets: 183


In [27]:
num_columns = ["Retail", "Wholesale", "Supply Volume"]

outliers = np.zeros(data.shape[0], dtype=bool)

for col in num_columns:
    z_scores = stats.zscore(data[col])
    outliers = outliers | (np.abs(z_scores) > 3) 

outlier_df = data[outliers].reset_index(drop=True)

data = data[~outliers]

In [28]:
len(outlier_df)

86

In [29]:
data = data.drop_duplicates()

In [30]:
data.to_csv("clean_data2", index = False)